<img src="./src/logo.png" width="250">

**Baustein:** Daten  $\rightarrow$ **Subbaustein:** Deskriptive Statistik, Visualisierung und Datenvorverarbeitung $\rightarrow$ **Übungsserie**

**Version:** 1.0, **Lizenz:** <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND 4.0</a>

***

# Neuronale Netzwerke: Grundlagen

---
## Importieren der notwendigen Python-Bibliotheken

In [ ]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

---
## Importieren der Daten

Im Folgenden laden wir den uns bekannten MNIST-Datensatz mit vorgefertigten Befehlen des PyTorch-Frameworks. Die Datensätze werden bei Bedarf automatisch aus dem Internet auf den JupyterHub heruntergeladen und als PyTorch-Objekt in den Arbeitsspeicher geladen.

In [ ]:
batch_size = 50

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

---
Wir können uns ein beliebiges (zum Beispiel das fünfte) Bild aus dem Trainingsdatensatz anzeigen lassen.

In [ ]:
plt.imshow(trainset[5][0][0], cmap="gray")

---
#### Aufgabe 1: Machen Sie sich mit der Variable `trainset` vertraut, indem Sie sich `trainset[0]`, `trainset[1]` und `trainset[2]` (also die ersten drei Trainingsbeispiele) ausgeben lassen. Was beinhalten diese Daten genau? Schreiben Sie eine Zeile Code, die das **Label** des 20-ten Trainingsbeispieles ausgibt.

---
## Definition eines Neuronalen Netzwerkes

Das Machine-Learning-Framework PyTorch erlaubt uns, Neuronale Netze im Handumdrehen zu definieren, ohne die einzelnen Berechnungen von Hand implementieren zu müssen. Im Folgenden definieren wir ein Neuronales Netzwerk, das drei Schichten hat:

- eine Input-Schicht mit 784 Input-Neuronen
- eine versteckte Schicht mit 300 Neuronen
- eine Output-Schicht mit 10 Neuronen

In [ ]:
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 300),
    nn.ReLU(),
    nn.Linear(300, 10),
)

---
#### Aufgabe 2: Warum muss die Input-Schicht genau 784 Inputs bzw. Neuronen haben?

Antwort:

#### Aufgabe 3: Warum muss die Output-Schicht genau 10 Outputs bzw. Neuronen haben?

Antwort:

---
## Der Training-Loop

Die Trainingschleife (oder Training-Loop) minimiert iterativ die Loss-Funktion bzw. den Trainingsverlust des Neuronalen Netzwerkes (siehe Vorlesung!). Dabei gibt es eine **äußere Schleife** (über die Epochen) und eine **innere Schleife** (über die Batches). Nach jedem Batch wird ein Gradientenschritt ausgeführt (`optimizer.step()`). Diese Berechnung wäre sehr aufwändig "from scratch" zu implementieren - zum Glück müssen wir das nicht. PyTorch bietet hierfür fertige Befehle, die maschinennah programmiert wurden. Genau dafür brauchen wir Machine-Learning-Frameworks.

In [ ]:
epochs = 10
learning_rate = 0.001

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

train_loss_history = np.array([])
test_loss_history = np.array([])

for epoch in range(epochs):

    running_loss = 0.0
    count = 0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        labels = nn.functional.one_hot(labels, num_classes=10).to(dtype=torch.float)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        count += 1

    train_loss_av = running_loss / count
    train_loss_history = np.append(train_loss_history, train_loss_av)
                
    with torch.no_grad():
        test_loss_av = 0.0
        count = 0
        for data in testloader:
            count += 1
            images, labels = data
            labels = nn.functional.one_hot(labels, num_classes=10).to(dtype=torch.float)
            outputs = net(images)
            test_loss = criterion(outputs, labels)
            test_loss_av += test_loss.item()
        
        test_loss_av = test_loss_av / count
        test_loss_history = np.append(test_loss_history, test_loss_av)
        
    print('[%d] train loss: %.3f,  test loss: %.3f' % (epoch, train_loss_av, test_loss_av))
    
    
print('Finished Training.')

plt.plot(train_loss_history)
plt.plot(test_loss_history)
plt.show()

#### Aufgabe 4: Wiederholen Sie die obenstehende Trainingsschleife, allerdings mit verschiedenen Modellarchitekturen. Variieren Sie dazu die Anzahl der Neuronen in der versteckten Schicht (aktuell 300) auf kleinere und größere Werte! Messen Sie dabei die Zeit des Trainingsverlaufes (mit Handy/Armbanduhr ist hier völlig ausreichend) und beobachten Sie den Verlauf der Loss-Kurven. Notieren und erklären Sie Ihre Beobachtungen.

Antwort:

#### Aufgabe 5: Wiederholen Sie die obenstehende Trainingsschleife, allerdings mit einem tieferen Modell. Fügen Sie dazu dem Netzwerk eine weitere versteckte Schicht mit 50 Neuronen hinzu. Messen Sie dabei die Zeit des Trainingsverlaufes (mit Handy/Armbanduhr ist hier völlig ausreichend) und beobachten Sie den Verlauf der Loss-Kurven. Notieren und erklären Sie Ihre Beobachtungen.

Antwort:

#### Aufgabe 6: Wiederholen Sie die obenstehende Trainingsschleife, allerdings mit verschiedenen Lernraten. Variieren Sie die Lernrate (aktuell 0.001) auf kleinere und größere Werte! Wie verhält sich der Trainingsverlauf? Notieren und erklären Sie Ihre Beobachtungen. 

Antwort:

---
## Auswertung

Zur Evaluation des Modells betrachten wir zuerst einmal eine kleine Teilmenge aus dem Testdatensatz.

In [ ]:
def imshow(img):
    npimg = img.cpu().numpy()
    plt.figure(figsize=(10,10))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
dataiter = iter(testloader)

images, labels = dataiter.next()
ncol=round(np.sqrt(1.618 * batch_size))


imshow(torchvision.utils.make_grid(images,nrow=ncol))

classes_of_labels = [classes[labels[i]] for i in range(batch_size)]
grouped = [classes_of_labels[i:i+ncol] for i in range(0, batch_size, ncol)]
print("Testlabels (Ground Truth):")
for l in grouped:
    print("".join("{:<6}".format(x) for x in l))

In [ ]:
outputs = net(images)

_, predicted = torch.max(outputs, 1)

classes_of_predictions = [classes[predicted[i]] for i in range(batch_size)]
grouped = [classes_of_predictions[i:i+ncol] for i in range(0, batch_size, ncol)]
print("Vorhersagen des Neuronalen Netzwerkes:")
for l in grouped:
    print("".join("{:<6}".format(x) for x in l))

---
Da der eine Batch aus dem Testdatensatz natürlich nicht hinreichend repräsentativ ist, betrachten wir jetzt den ganzen Testdatensatz und werten diesen in einer Wahrheitsmatrix ("Confusion Matrix") aus. Hierbei werden alle Klassifikationen einander gegenübergestellt und gezählt ("Welche Ziffer wurde wie oft als was klassifiziert?").

In [ ]:
M = len(classes)

conf_mat = torch.zeros((M,M), dtype=torch.int32)

with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = nn.functional.one_hot(labels, num_classes=M).to(dtype=torch.float)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        images, labels = data
        for label, prediction in zip(labels, predictions):
            conf_mat[label, prediction] += 1
            


disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat.numpy(),display_labels=classes)
disp.plot(xticks_rotation=45.)
plt.show()

#### Aufgabe 7: Interpretieren Sie die Confusion Matrix. Welche Ziffern werden besonders gut, welche besonders schlecht klassifiziert? Geben Sie an, welche die häufigste Verwechslung ist und stellen sie eine Vermutung an, wieso.

Antwort:

#### Aufgabe 8: Schreiben Sie ein bis fünf Zeilen Code, in denen Sie aus der Confusion Matrix (Variable `conf_mat`) die Accuracy des Neuronalen Netzwerkes berechnen. Reminder: Die Accuracy ist gleich der Summe aller Diagonalelemente geteilt durch die Summe aller Elemente der Matrix.

In [ ]:
### Ihr Code hier!

print(accuracy)

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Die Übungsserie begleitend zum AI4ALL-Kurs</span> der <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">EAH Jena</span> ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Namensnennung - Nicht kommerziell - Keine Bearbeitungen 4.0 International Lizenz</a>.

Der AI4ALL-Kurs entsteht im Rahmen des Projekts MoVeKI2EAH. Das Projekt MoVeKI2EAH wird durch das BMBF (Bundesministerium für Bildung und Forschung) und den Freistaat Thüringen im Rahmen der Bund-Länder-Initiative zur Förderung von Künstlicher Intelligenz in der Hochschulbildung gefördert (12/2021 bis 11/2025, Föderkennzeichen 16DHBKI081).